In [3]:
from src.preprocessor.utils.dataset_level import save_pickle

corpus = save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/corpus.pkl')
queries = save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/queries.pkl')
relevant_docs = save_pickle('/home/thiendc/projects/legal_retrieval/data/processed/relevant_docs.pkl')

corpus

{'0': 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.',
 '1': 'Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân cấp xã nơi có đê phải tổ chức lực lượng lao động tại địa phương để tuần tra, canh gác đê và thường trực trên các điếm canh đê hoặc nhà dân khu vực gần đê (đối với những khu vực chưa có điếm canh đê), khi có báo động lũ từ cấp I trở lên đối với tuyến sông có đê (sau đây gọi tắt là lực lượng tuần tra, canh gác đê). Lực lượng tuần tra, canh gác đê được tổ chức thành các đội, do Ủy ban nhân dân cấp xã ra quyết định thành lập; từ 01 đến 02 kilômét đê thành lập 01 đội; mỗi đội có từ 12 đến 18 người, trong đó có 01 đội trưởng và 01 hoặc 02 đội phó. Danh sách thành viên đội tuần tra, canh gác đê được niêm yết tại điếm canh đê thuộc địa bàn được phân công. Khi lũ, bão có diễn biến phức tạp, kéo dài ngày, Uỷ ban nhân dân cấp xã có thể quyết định việc bổ sung thêm thành viên c